<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TF

Sequence-to-sequence
* One to one- Image Captioning
* Many to one- Sentiment analysis
* Many to one- time series forecasting
* many- to-many- Machine translation


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-81774efe-526a-c960-af6b-2fe403438fc6)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-14 04:28:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-14 04:28:52 (102 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-06-14 04:28:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.128, 173.194.69.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.29MB/s    in 0.4s    

2023-06-14 04:28:56 (1.29 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

To visualize the sample data, we need to read the in. On way is to use python.

Another way to do this is use pandas



In [8]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [10]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
train_data_shuffled=train_df.sample(frac=1, random_state=42)

In [14]:
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [15]:
#Test dataframe looks like:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [18]:
#How many total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [22]:
#Let's visualize some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
text:
Soul food sound so bomb right now '

---

Target: 0 (not real disaster)
text:
Truth...
https://t.co/beJfTYgJIL
#News
#BBC
#CNN
#Islam
#Truth
#god
#ISIS
#terrorism
#Quran
#Lies http://t.co/jlCZiDZ7Vu

---

Target: 0 (not real disaster)
text:
Û¢Û¢If your lost &amp; alone or your sinking like a stone carry onå¡å¡

---

Target: 1 (real disaster)
text:
@mylittlepwnies3 @Early__May @AnathemaZhiv @TonySandos much of which has to do with lebanon 80s attack/ iran hostage crisis/ Libya Pan am

---

Target: 1 (real disaster)
text:
California police officer pulls gun on man without any apparent provocation (VIDEO) http://t.co/Lhw4vTbHZG via @dailykos

---

